# The North Face ecommerce project


Contents
--------
1. [Loading the dataset](#loading)
2. [Preliminary EDA](#eda)
2. [Conclusion and perspectives](#conclusion)



In [ ]:
import csv
import re
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import wordcloud

## <a id="loading"></a> Data loading and preprocessing

download with `python -m spacy download en_core_web_sm`

- remove stop words
- remove punctuation
- normalize to lowercase

In [ ]:
## Load the file
with open('../sample-data.csv', 'rt', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',', quoting=csv.QUOTE_STRINGS)
    next(reader, None) # remove header
    descriptions = [row[1] for row in reader]

In [ ]:
## remove HTML tags
descriptions = [re.sub(r'<[a-z/]+>', ' ', d) for d in descriptions]

In [ ]:
## normalize into lemmas
nlp = spacy.load('en_core_web_sm')
lemma_corpus = []
for desc in descriptions:
    doc = nlp(desc)
    doc = [wd.lemma_.lower() for wd in doc
           if not wd.is_stop and (wd.like_num or wd.is_alpha)]
    lemma_corpus.append(np.array(doc))